# **Parcial 2: Prediciendo el PIB por medio de redes neuronales**

**Universidad de los Andes**

Integrantes:

*   Marcelo Yepes

*   Angela Sofia Torres

*   Jennifer Paola Sarabia

*  Laura Pabón

*   Dario Montoya

# **Instalación de Bibliotecas y Carga del Dataset**

In [ ]:
#Instalación de las librerías
!pip install -q pandas
!pip install -q numpy
!pip install -q scipy
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q plotly
!pip install -q yellowbrick
!pip install -q scikit-learn
!pip install -q imbalanced-learn
!pip install -q tqdm
!pip install -q joblib
!pip install -q huggingface_hub
!pip install -q datasets
!pip install -q ydata_profiling

**Librerías de Machine Learning**

*   ColumnTransformer: Para aplicar diferentes transformaciones a diferentes columnas.
*  train_test_split: Divide los datos en entrenamiento y prueba.
*   Escaladores (MinMaxScaler, StandardScaler): Normalización de datos.
*   Codificadores (LabelEncoder, OneHotEncoder): Convierte datos categóricos en numéricos.
*   Métricas: Evalúan el rendimiento de los modelos.

In [ ]:
#Importar las librerías necesarias y definir semilla
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import files
from ydata_profiling import ProfileReport
from numpy import sqrt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report, ConfusionMatrixDisplay

# Quitar el límite de columnas a mostrar en un DataFrame.
pd.set_option('display.max_columns', None)

# Fijar semilla para reproducibilidad
SEED = 2025

In [ ]:
!kaggle datasets download sazidthe1/world-gdp-data

In [ ]:
DATASET_NAME = 'world-gdp-data'
!unzip -o {DATASET_NAME}.zip -d {DATASET_NAME}


# **1. Preparación y procesamiento de los datos**

## **Juntar las bases de datos de acuerdo con el código del país**

In [ ]:
# Listar archivos en el directorio actual
print(os.listdir("world-gdp-data"))

# Cargar los datasets en formato csv
gdp_df = pd.read_csv("/content/world-gdp-data/gdp_data.csv")
country_df = pd.read_csv("/content/world-gdp-data/country_codes.csv")


# Verificar contenido antes de la unión
print("GDP Dataset:")
display(gdp_df.head())

print("\nCountry Codes Dataset:")
display(country_df.head())

In [ ]:
# Unir los datasets usando la columna 'country_code'
df_merged= pd.merge(gdp_df, country_df, on="country_code", how="inner")

df_merged = df_merged.rename(columns={"value": "gdp"})

# Mostrar las filas del dataset combinado
df_merged

## Bono: añadir una nueva variable

Se tomó la variable de porcentaje de inflación anual para 266 países con datos desde 1960 hasta 2022. Esta base de datos se tomó del Banco Mundial y se encuentra en este enlace: https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?view=chart

Se escogió tomar la variable de inflación ya que se cree que niveles altos de inflación tiene una relación negativa sobre el PIB. La inlación indica los aumentos generalizados de precios, lo cual reduce el poder adquisitivo y puede contraer el consumo y la inversión, componentes cruciales del PIB. Así una inflación alta y volátil puede desacelerar el crecimiento económico, afectando negativamente la competitividad y el comercio exterior de un país, lo que se puede asociar con un menor PIB. Por este motivo, consideramos que la variables de inflación puede ser útil para predecir el nivel del PIB de cada país.

In [ ]:
#Cargar la base de datos de inflación manualmente
df_inflacion = pd.read_csv("/content/inflacion_data.csv", sep=",", skiprows=3)

df_inflacion.info()

df_inflacion = df_inflacion.drop(columns=['Country Name', 'Indicator Name', 'Indicator Code', '2023', 'Unnamed: 68'])

df_inflacion

In [ ]:
#Convertir los años en filas para que quede en el mismo formato que gdp_df

df_inflacion_long = df_inflacion.melt(id_vars=["Country Code"], var_name="year", value_name="inflation")
df_inflacion_long["year"] = df_inflacion_long["year"].astype(int)
df_inflacion_long = df_inflacion_long.rename(columns={"Country Code": "country_code"})

df_inflacion_long

In [ ]:
#Unir el merge realizado anteriormente con los datos del PIB con la base de datos de inflación usando el código dle país
df = df_merged.merge(df_inflacion_long, on=["country_code", "year"], how="right")

df

##**Explorar el dataset**

**Diccionario de variables**

| **Variable**   | **Descripción**                                                                                  | **Tipo de dato** |
|---------------|--------------------------------------------------------------------------------------------------|------------------|
| country_name  | Nombre del país.                                                                                 | Texto           |
| country_code  | Código del país según la norma (código de tres letras).                        | Texto           |
| year          | Año al que corresponde el dato.                                                          | Numérico        |
| value         | Valor del PIB en dólares estadounidenses actuales.                                               | Numérico        |
| region        | Región geográfica a la que pertenece el país (por ejemplo, Asia, Europa).                        | Texto           |
| income_group  | Clasificación del país según su nivel de ingresos (por ejemplo, ingresos altos, medios, bajos). | Texto           |
| inflation     | Tasa de inflación anual (%), basada en el índice de precios al consumidor.                        | Numérico        |


In [ ]:
# Información general de los datasets
print(gdp_df.info())
print(country_df.info())
print(df_inflacion.info())

# Estadísticas descriptivas
print ("Estadísticas descriptivas de la base de datos")
print(df.describe())


## **Clasificación de variables y preparación**

In [ ]:
# Hacer Label Encoder para la variable 'income_group'

# Crear un diccionario con los valores personalizados
income_mapping = {
    "High income": 4,
    "Upper middle income": 3,
    "Lower middle income": 2,
    "Low income": 1
}

# Aplicar el mapeo a la columna 'income_group'
df['income_group'] = df['income_group'].map(income_mapping)

# Aplicar One-Hot Encoding a 'region'
df_encoded = pd.get_dummies(df, columns=['region'])
df_encoded

In [ ]:
# Para transformar la variable categórica de los países

# Modificar el nombre de los paises por un ranking
country_gpd = df_encoded.groupby('country_name')['gdp'].mean().sort_values()
country_ranking = {country: rank for rank, country in enumerate(country_gpd.index)}

df_encoded['country_name'] = df_encoded['country_name'].map(country_ranking)

df_encoded.head()

In [ ]:
#Valores únicos en la variable año
df_encoded['year'].unique()

#Transformación de los años en columnas

df_final = df.pivot(index="country_code", columns="year", values=["gdp", "inflation"])

df_final.columns[-64]

df_final


In [ ]:
#Verificar valor NA en la base de datos
print(df_final.isna().sum())

# Reemplazar todos los NaN en el DataFrame con -1
df_final.fillna(-1, inplace=True)

print(df_final.isna().sum())

In [ ]:
# Verificación de valores nulos
print(df_final.isnull().sum())

**Crear la variable objetivo (GDP para el año 2022)**

In [ ]:
df_final.columns[-64]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Definir el nombre de la columna correspondiente al PIB del 2022
gdp_2022 = ('gdp', 2022)
gdp_2021 = ('gdp', 2021)

# Reemplazar NaN en la columna 2022 con los valores de la columna 2021
df_final[gdp_2022] = df_final[gdp_2022].fillna(df_final[gdp_2021])

# Calcular percentiles SOLO sobre la columna 2022
low_threshold = np.percentile(df_final[gdp_2022].dropna(), 33)  # 33% de los datos
high_threshold = np.percentile(df_final[gdp_2022].dropna(), 66)  # 66% de los datos

# Función para asignar la categoría basada en los percentiles de 2022
def categorize_gdp(value):
    if value <= low_threshold:
        return "Low GDP"
    elif value <= high_threshold:
        return "Medium GDP"
    else:
        return "High GDP"

# Aplicar la función de categorización sobre la columna GDP_2022
df_final["GDP_Level"] = df_final[gdp_2022].apply(categorize_gdp)

# Mostrar la distribución de clases
print(df_final["GDP_Level"].value_counts())

# Codificar la variable categórica con LabelEncoder
le = LabelEncoder()
df_final["GDP_Level"] = le.fit_transform(df_final["GDP_Level"])

df_final.drop(columns=[gdp_2022], inplace=True)

# Mostrar primeras filas con la variable de clasificación
display(df_final.head())

## **División de los datos en test y train**

In [ ]:
# Separar características (X) y variable objetivo (y)
X = df_final.drop(columns=["GDP_Level"])
y = df_final["GDP_Level"]

# Dividir en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Mostrar dimensiones de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

In [ ]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

### **Análisis descriptivo de los datos de train y test**

#### **Análisis datos train**

In [ ]:
# Reporte análisis descriptivo de los datos de entrenamiento
reporte_train = ProfileReport(df_train, title="Profiling Report Train dataset", minimal=True)
reporte_train.to_file("reporte_train.html")
reporte_train

#### **Análisis datos test**

In [ ]:
# Reporte análisis descriptivo de los datos de entrenamiento
reporte_test = ProfileReport(df_train, title="Profiling Report Test dataset", minimal=True)
reporte_test.to_file("reporte_test.html")
reporte_test

# **2. Construcción modelos redes neuronales**

## 2. a. Modelo de red neuronal tradicional (Scikit-Learn)

In [ ]:
#Estandarizar los datos de X de la base detos

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report


# 📌 Guardar nombres de columnas y el índice antes de escalar
column_names = X_train.columns
index_train = X_train.index
index_test = X_test.index

# 📌 Escalar los datos después de la limpieza
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 📌 Convertir de nuevo a DataFrame manteniendo nombres de columnas e índice
X_train_scaled = pd.DataFrame(X_train_scaled, columns=column_names, index=index_train)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=column_names, index=index_test)

display(X_train_scaled.head())

### Búsqueda de hiperparámetros para modelo tradicional

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Definir los hiperparámetros a optimizar con solo una capa oculta
param_grid = {
    'hidden_layer_sizes': [(16,), (32,), (64,), (128,)],  # Diferentes tamaños para una única capa oculta
    'activation': ['relu', 'tanh', 'logistic'],  # Función de activación
    'solver': ['adam', 'lbfgs'],  # Métodos de optimización
    'alpha': [0.0001, 0.01, 0.1],  # Regularización L2
    'learning_rate_init': [0.001, 0.01, 0.05],  # Tasa de aprendizaje inicial
    'batch_size': [32, 64, 'auto'],  # Tamaño del lote
}

# Definir el modelo base
mlp = MLPClassifier(max_iter=200, random_state=42)

# Configurar Grid Search
grid_search = GridSearchCV(mlp, param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# Ejecutar la búsqueda de hiperparámetros
grid_search.fit(X_train_scaled, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("\n🔍 Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Entrenar modelo con los mejores hiperparámetros
best_mlp = grid_search.best_estimator_

# Evaluar en el conjunto de prueba
y_pred = best_mlp.predict(X_test_scaled)
print("\n✅ Resultados del mejor modelo:")
print(classification_report(y_test, y_pred))

In [ ]:
#  PARA TEST
#Paso 1: Hacer predicciones
y_pred = best_mlp.predict(X_test_scaled)

#  Paso 2: Imprimir reporte de clasificación
print("\n✅ Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

#  Paso 3: Graficar la Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Low", "Medium", "High"], yticklabels=["Low", "Medium", "High"])
plt.xlabel("Predicciones")
plt.ylabel("Valores Reales")
plt.title("Matriz de Confusión - MLPClassifier Test")
plt.show()

#  PARA TRAIN
#Paso 1: Hacer predicciones
y_pred = best_mlp.predict(X_train_scaled)

#  Paso 2: Imprimir reporte de clasificación
print("\n✅ Reporte de Clasificación:")
print(classification_report(y_train, y_pred))

#  Paso 3: Graficar la Matriz de Confusión
conf_matrix = confusion_matrix(y_train, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Low", "Medium", "High"], yticklabels=["Low", "Medium", "High"])
plt.xlabel("Predicciones")
plt.ylabel("Valores Reales")
plt.title("Matriz de Confusión - MLPClassifier Train")
plt.show()


In [ ]:
# Gráfica función de pérdida por iteración
import matplotlib.pyplot as plt

# 📌 Verificar si el modelo ha almacenado el historial de pérdidas
if hasattr(best_mlp, 'loss_curve_'):
    plt.plot(best_mlp.loss_curve_, label='Loss durante entrenamiento')
    plt.xlabel('Iteraciones (Épocas)')
    plt.ylabel('Pérdida (Loss)')
    plt.title('Evolución de la Pérdida durante el Entrenamiento')
    plt.legend()
    plt.grid()
    plt.show()
else:
    print("El modelo no ha almacenado la curva de pérdida.")

#Gráfica tasa de aprendizaje
import numpy as np
import matplotlib.pyplot as plt

# 📌 Definir la tasa de aprendizaje inicial (se obtiene de los hiperparámetros del mejor modelo)
learning_rate_init = best_mlp.learning_rate_init

# 📌 Simular la evolución de la tasa de aprendizaje si es 'adaptive'
if best_mlp.learning_rate == 'adaptive':
    learning_rates = [learning_rate_init / (1.0 + 0.1 * i) for i in range(len(best_mlp.loss_curve_))]
else:
    learning_rates = [learning_rate_init] * len(best_mlp.loss_curve_)  # Tasa de aprendizaje fija

# Graficar la tasa de aprendizaje
plt.figure(figsize=(8, 5))
plt.plot(learning_rates, label="Tasa de Aprendizaje", color='red')
plt.xlabel('Iteraciones (Épocas)')
plt.ylabel('Tasa de Aprendizaje')
plt.title('Evolución de la Tasa de Aprendizaje durante el Entrenamiento')
plt.legend()
plt.grid()
plt.show()

#Curva ROC
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

# 📌 Binarizar las clases de y_test para el cálculo de la ROC multiclase
n_classes = len(np.unique(y_test))  # Número de clases únicas
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))  # Convierte en formato binarizado
y_probs = best_mlp.predict_proba(X_test_scaled)  # Probabilidades de cada clase

# 📌 Calcular ROC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

plt.figure(figsize=(10, 5))

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_probs[:, i])  # ROC para cada clase
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], lw=2, label=f'Clase {i} (AUC = {roc_auc[i]:.2f})')

# 📌 Agregar línea de referencia y etiquetas
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC Multiclase')
plt.legend(loc="lower right")

# Gráfica de precisión

Gráfica precisión del modelo

In [ ]:
import matplotlib.pyplot as plt

# 📌 Verificar si el modelo tiene historial de entrenamiento
if hasattr(best_mlp, "loss_curve_"):
    plt.plot(best_mlp.loss_curve_, label="Pérdida en entrenamiento", color="b")

    plt.xlabel("Iteraciones (Épocas)")
    plt.ylabel("Pérdida (Loss)")
    plt.title("Evolución de la Pérdida del Modelo")
    plt.legend()
    plt.grid()
    plt.show()
else:
    print("⚠️ No hay historial de pérdida disponible. Verifica que el modelo haya sido entrenado correctamente.")


In [ ]:
#Para guardar el modelo

import joblib

# Guardar modelo de Scikit-Learn
joblib.dump(best_mlp, "modelo_sklearn.pkl")

# Cargar modelo después
modelo_sklearn = joblib.load("modelo_sklearn.pkl")


## 2. b. Modelo red neuronal profunda (Tensorflow)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 📌 Paso 1: Definir la arquitectura de la Red Neuronal Profunda
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Capa de entrada
    Dropout(0.2),  # Regularización
    Dense(64, activation='relu'),  # Primera capa oculta
    Dropout(0.2),
    Dense(32, activation='relu'),  # Segunda capa oculta
    Dense(3, activation='softmax')  # Capa de salida (3 clases: Low, Medium, High GDP)
])

# 📌 Paso 2: Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),  # Optimizador Adam
              loss='sparse_categorical_crossentropy',  # Función de pérdida
              metrics=['accuracy'])

# 📌 Paso 3: Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

# 📌 Paso 4: Evaluación final
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n📊 Precisión en el conjunto de prueba: {test_acc:.4f}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

#PARA TEST
# 📌 Hacer predicciones
y_pred_nn = model.predict(X_test_scaled)
y_pred_nn = y_pred_nn.argmax(axis=1)  # Convertir probabilidades en clases

# 📌 Imprimir reporte de clasificación
print("\n✅ Reporte de Clasificación - Red Neuronal TensorFlow:")
print(classification_report(y_test, y_pred_nn))

# 📌 Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred_nn)

plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Low", "Medium", "High"], yticklabels=["Low", "Medium", "High"])
plt.xlabel("Predicciones")
plt.ylabel("Valores Reales")
plt.title("Matriz de Confusión - TensorFlow Test")
plt.show()

#PARA TRAIN
# 📌 Hacer predicciones
y_pred_nn = model.predict(X_train_scaled)
y_pred_nn = y_pred_nn.argmax(axis=1)  # Convertir probabilidades en clases

# 📌 Imprimir reporte de clasificación
print("\n✅ Reporte de Clasificación - Red Neuronal TensorFlow:")
print(classification_report(y_train, y_pred_nn))

# 📌 Matriz de Confusión
conf_matrix = confusion_matrix(y_train, y_pred_nn)

plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Low", "Medium", "High"], yticklabels=["Low", "Medium", "High"])
plt.xlabel("Predicciones")
plt.ylabel("Valores Reales")
plt.title("Matriz de Confusión - TensorFlow Train")
plt.show()


In [ ]:
#Gráfica tasa de aprendizaje
import matplotlib.pyplot as plt

# Extraer la tasa de aprendizaje de cada época desde el optimizador
learning_rates = [model.optimizer.learning_rate.numpy() for _ in history.epoch]

# Graficar la evolución de la tasa de aprendizaje
plt.plot(history.epoch, learning_rates, label="Tasa de Aprendizaje")
plt.xlabel("Épocas")
plt.ylabel("Learning Rate")
plt.title("Evolución de la Tasa de Aprendizaje")
plt.legend()
plt.grid()
plt.show()


# Pérdida del modelo
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Evolución de la función de pérdida modelo redes neuronales profundas")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.legend()
plt.show()

# Precisión en entrenamiento y validación
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title("Evolución de la precisión modelo redes neuronales profundas")
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.legend()
plt.show()

# Suponiendo que entrenaste el modelo con history = modelo.fit(...)
plt.plot(history.history['loss'], label='Pérdida en entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en validación')

plt.xlabel('Iteraciones (Épocas)')
plt.ylabel('Pérdida')
plt.title('Tasa de Aprendizaje vs Iteraciones')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Para guardar el modelo

from tensorflow import keras

# Guardar modelo de TensorFlow
modelo_tf.save("modelo_tensorflow.keras")

# Cargar modelo después
modelo_tensorflow = keras.models.load_model("modelo_tensorflow.keras")

## 3. Red neuronal con mala función de pérdida

In [ ]:
from tensorflow.keras.optimizers import SGD

# 📌 Paso 1: Definir la arquitectura del modelo con errores
model_bad = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Capa de entrada
    Dense(64, activation='relu'),  # Primera capa oculta (sin Dropout)
    Dense(32, activation='relu'),  # Segunda capa oculta (sin BatchNorm)
    Dense(3, activation='softmax')  # Capa de salida (3 clases)
])


# 📌 Paso 2: Compilar con errores
model_bad.compile(optimizer=SGD(learning_rate=1.0),  # Learning rate demasiado alto
                  loss='sparse_categorical_crossentropy',  # Función de pérdida incorrecta (debería ser 'sparse_categorical_crossentropy')
                  metrics=['accuracy'])

# 📌 Paso 3: Entrenar el modelo
history_bad = model_bad.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_data=(X_test_scaled, y_test))

# 📌 Paso 4: Evaluación del modelo
test_loss_bad, test_acc_bad = model_bad.evaluate(X_test_scaled, y_test)
print(f"\n❌ Precisión en test (modelo erróneo): {test_acc_bad:.4f}")



In [ ]:
# 📌 Gráfica de la pérdida (se espera que sea inestable)
plt.plot(history_bad.history['loss'], label='Entrenamiento')
plt.plot(history_bad.history['val_loss'], label='Validación')
plt.title("🚨 Pérdida en el modelo mal configurado")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.legend()
plt.show()

# 📌 Gráfica de la precisión (se espera que no mejore mucho)
plt.plot(history_bad.history['accuracy'], label='Entrenamiento')
plt.plot(history_bad.history['val_accuracy'], label='Validación')
plt.title("🚨 Precisión en el modelo mal configurado")
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.legend()
plt.show()


In [ ]:
# 📌 Hacer predicciones con el modelo mal configurado
y_pred_bad = model_bad.predict(X_test_scaled)
y_pred_bad = y_pred_bad.argmax(axis=1)  # Convertir probabilidades en clases

# 📌 Matriz de confusión
conf_matrix_bad = confusion_matrix(y_test, y_pred_bad)

plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix_bad, annot=True, fmt='d', cmap='Reds', xticklabels=["Low", "Medium", "High"], yticklabels=["Low", "Medium", "High"])
plt.xlabel("Predicciones")
plt.ylabel("Valores Reales")
plt.title("🚨 Matriz de Confusión - Modelo Mal Configurado")
plt.show()
